In [3]:
# flake8: noqa: E501
#
# En este dataset se desea pronosticar el default (pago) del cliente el próximo
# mes a partir de 23 variables explicativas.
#
#   LIMIT_BAL: Monto del credito otorgado. Incluye el credito individual y el
#              credito familiar (suplementario).
#         SEX: Genero (1=male; 2=female).
#   EDUCATION: Educacion (0=N/A; 1=graduate school; 2=university; 3=high school; 4=others).
#    MARRIAGE: Estado civil (0=N/A; 1=married; 2=single; 3=others).
#         AGE: Edad (years).
#       PAY_0: Historia de pagos pasados. Estado del pago en septiembre, 2005.
#       PAY_2: Historia de pagos pasados. Estado del pago en agosto, 2005.
#       PAY_3: Historia de pagos pasados. Estado del pago en julio, 2005.
#       PAY_4: Historia de pagos pasados. Estado del pago en junio, 2005.
#       PAY_5: Historia de pagos pasados. Estado del pago en mayo, 2005.
#       PAY_6: Historia de pagos pasados. Estado del pago en abril, 2005.
#   BILL_AMT1: Historia de pagos pasados. Monto a pagar en septiembre, 2005.
#   BILL_AMT2: Historia de pagos pasados. Monto a pagar en agosto, 2005.
#   BILL_AMT3: Historia de pagos pasados. Monto a pagar en julio, 2005.
#   BILL_AMT4: Historia de pagos pasados. Monto a pagar en junio, 2005.
#   BILL_AMT5: Historia de pagos pasados. Monto a pagar en mayo, 2005.
#   BILL_AMT6: Historia de pagos pasados. Monto a pagar en abril, 2005.
#    PAY_AMT1: Historia de pagos pasados. Monto pagado en septiembre, 2005.
#    PAY_AMT2: Historia de pagos pasados. Monto pagado en agosto, 2005.
#    PAY_AMT3: Historia de pagos pasados. Monto pagado en julio, 2005.
#    PAY_AMT4: Historia de pagos pasados. Monto pagado en junio, 2005.
#    PAY_AMT5: Historia de pagos pasados. Monto pagado en mayo, 2005.
#    PAY_AMT6: Historia de pagos pasados. Monto pagado en abril, 2005.
#
# La variable "default payment next month" corresponde a la variable objetivo.
#
# El dataset ya se encuentra dividido en conjuntos de entrenamiento y prueba
# en la carpeta "files/input/".
#
# Los pasos que debe seguir para la construcción de un modelo de
# clasificación están descritos a continuación.
#
#
# Paso 1.
# Realice la limpieza de los datasets:
# - Renombre la columna "default payment next month" a "default".
# - Remueva la columna "ID".
# - Elimine los registros con informacion no disponible.
# - Para la columna EDUCATION, valores > 4 indican niveles superiores
#   de educación, agrupe estos valores en la categoría "others".
# - Renombre la columna "default payment next month" a "default"
# - Remueva la columna "ID".


import zipfile
import pickle
import gzip
import json
import os
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def clean_dataset(df):
    """
    Limpia el dataset para el modelo de clasificación.
    - Renombra la columna objetivo a 'default'.
    - Elimina la columna 'ID'.
    - Elimina registros con valores faltantes.
    - Elimina registros donde EDUCATION o MARRIAGE son cero.
    - Agrupa EDUCATION > 4 como categoría 'others' (valor = 4).
    """
    df_clean = df.drop(columns=["ID"], errors="ignore")  # Evita error si no existe 'ID'

    if "default payment next month" in df_clean.columns:
        df_clean = df_clean.rename(columns={"default payment next month": "default"})

    df_clean = df_clean.dropna()

    # Filtrado extra: EDUCATION y MARRIAGE no pueden ser cero
    df_clean = df_clean[(df_clean["EDUCATION"] != 0) & (df_clean["MARRIAGE"] != 0)]

    # Agrupamos niveles altos de EDUCATION
    df_clean.loc[df_clean["EDUCATION"] > 4, "EDUCATION"] = 4

    return df_clean

def process_zip(zip_path, output_name):
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_contents = zip_ref.namelist()
        print(f"Contenido del zip: {zip_contents}")
        csv_name = zip_contents[0]  # Asumimos que hay solo un CSV
        zip_ref.extract(csv_name, "files/input/")

    df = pd.read_csv(os.path.join("files/input/", csv_name))
    df_clean = clean_dataset(df)

    os.makedirs(os.path.dirname(output_name), exist_ok=True)  # Crear carpeta si no existe
    df_clean.to_csv(output_name, index=False)
    print(f"Limpieza completada: {output_name}")

# Ejecutar
process_zip("../files/input/train_data.csv.zip", "../files/input/train_data_clean.csv")
process_zip("../files/input/test_data.csv.zip", "../files/input/test_data_clean.csv")




Contenido del zip: ['train_default_of_credit_card_clients.csv']
Limpieza completada: ../files/input/train_data_clean.csv
Contenido del zip: ['test_default_of_credit_card_clients.csv']
Limpieza completada: ../files/input/test_data_clean.csv


In [4]:
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.


# Cargar datasets limpios
train_df = pd.read_csv("../files/input/train_data_clean.csv")
test_df = pd.read_csv("../files/input/test_data_clean.csv")

# Separar variables predictoras (X) y objetivo (y)
X_train = train_df.drop(columns=["default"])
y_train = train_df["default"]

X_test = test_df.drop(columns=["default"])
y_test = test_df["default"]

# Verificar shapes
print("Shapes:")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)


Shapes:
X_train: (20953, 23)
y_train: (20953,)
X_test: (8979, 23)
y_test: (8979,)


In [5]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Descompone la matriz de entrada usando PCA. El PCA usa todas las componentes.
# - Estandariza la matriz de entrada.
# - Selecciona las K columnas mas relevantes de la matrix de entrada.
# - Ajusta una maquina de vectores de soporte (svm).

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC

# Identificar columnas categóricas y numéricas
categorical_cols = ["SEX", "EDUCATION", "MARRIAGE"]
numerical_cols = [col for col in X_train.columns if col not in categorical_cols]

# Pipeline para variables categóricas
categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Pipeline para variables numéricas
numerical_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())  # Estandarizar numéricas
])

# Combinar transformaciones
preprocessor = ColumnTransformer(transformers=[
    ("cat", categorical_transformer, categorical_cols),
    ("num", numerical_transformer, numerical_cols)
])

# Pipeline completo
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("pca", PCA()),  # Todas las componentes
    ("select", SelectKBest(score_func=f_classif)),  # Selección de K mejores variables
    ("svm", SVC())  # Modelo SVM
])



In [20]:
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use la función de precision
# balanceada para medir la precisión del modelo.

from sklearn.model_selection import GridSearchCV

# Definir hiperparámetros a probar
param_grid = {
    'pca__n_components': [20, 21],  
    'select__k': [12],                 
    'svm__kernel': ['rbf'],         
    'svm__gamma': [0.099]           
}

# Crear GridSearchCV envolviendo el pipeline
best_pipeline = GridSearchCV(
    estimator=pipeline,         # pipeline del Paso 3
    param_grid=param_grid,
    cv=10,                      # validación cruzada con 10 splits
    scoring="balanced_accuracy",
    n_jobs=-1,                  # usar todos los cores
    verbose=1,
    return_train_score=True
)

# Entrenar GridSearchCV
best_pipeline.fit(X_train, y_train)

print(f"✅ Mejor combinación de parámetros: {best_pipeline.best_params_}")
print(f"✅ Mejor puntuación balanced accuracy: {best_pipeline.best_score_:.4f}")


Fitting 10 folds for each of 2 candidates, totalling 20 fits
✅ Mejor combinación de parámetros: {'pca__n_components': 20, 'select__k': 12, 'svm__gamma': 0.099, 'svm__kernel': 'rbf'}
✅ Mejor puntuación balanced accuracy: 0.6502


In [27]:
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

import gzip
import pickle
import os

# Crear carpeta si no existe
os.makedirs("../files/models", exist_ok=True)

# Ruta del modelo comprimido
model_path = "../files/models/model.pkl.gz"

# Guardar modelo comprimido
with gzip.open(model_path, "wb") as f:
    pickle.dump(best_pipeline, f)

print(f"✅ Modelo guardado correctamente en: {model_path}")

with gzip.open("../files/models/model.pkl.gz", "rb") as f:
    loaded_model = pickle.load(f)

# Usar el modelo cargado para predecir
y_pred = loaded_model.predict(X_test)

import gzip
import pickle

def load_model(path):
    """Carga un modelo previamente guardado."""
    with gzip.open(path, "rb") as f:
        model = pickle.load(f)
    return model

model = load_model("../files/models/model.pkl.gz")

# Usar para predecir
y_pred = model.predict(X_test)
model = load_model("../files/models/model.pkl.gz")




✅ Modelo guardado correctamente en: ../files/models/model.pkl.gz


In [29]:
# Paso 6.
# Calcule las metricas de precision, precision balanceada, recall,
# y f1-score para los conjuntos de entrenamiento y prueba.
# Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# Este diccionario tiene un campo para indicar si es el conjunto
# de entrenamiento o prueba. Por ejemplo:
#
# {'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
# {'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}

from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score
import json
import os

os.makedirs("../files/output", exist_ok=True)
metrics_path = "../files/output/metrics.json"

# Predicciones
y_train_pred = best_pipeline.predict(X_train)
y_test_pred = best_pipeline.predict(X_test)

# Métricas
metrics_train = {
    "type": "metrics",
    "dataset": "train",
    "precision": precision_score(y_train, y_train_pred),
    "balanced_accuracy": balanced_accuracy_score(y_train, y_train_pred),
    "recall": recall_score(y_train, y_train_pred),
    "f1_score": f1_score(y_train, y_train_pred)
}

metrics_test = {
    "type": "metrics",
    "dataset": "test",
    "precision": precision_score(y_test, y_test_pred),
    "balanced_accuracy": balanced_accuracy_score(y_test, y_test_pred),
    "recall": recall_score(y_test, y_test_pred),
    "f1_score": f1_score(y_test, y_test_pred)
}

# Guardar métricas en formato JSON Lines
with open(metrics_path, "w") as f:
    f.write(json.dumps(metrics_train) + "\n")
    f.write(json.dumps(metrics_test) + "\n")

print(f"✅ Métricas guardadas en: {metrics_path}")


✅ Métricas guardadas en: ../files/output/metrics.json


In [31]:
# Paso 7.
# Calcule las matrices de confusión para los conjuntos de entrenamiento y prueba.
# Guardelas en el archivo files/output/metrics.json.


from sklearn.metrics import confusion_matrix

cm_train = confusion_matrix(y_train, y_train_pred)
cm_test = confusion_matrix(y_test, y_test_pred)

cm_train_dict = {
    "type": "cm_matrix",
    "dataset": "train",
    "true_0": {"predicted_0": int(cm_train[0][0]), "predicted_1": int(cm_train[0][1])},
    "true_1": {"predicted_0": int(cm_train[1][0]), "predicted_1": int(cm_train[1][1])}
}

cm_test_dict = {
    "type": "cm_matrix",
    "dataset": "test",
    "true_0": {"predicted_0": int(cm_test[0][0]), "predicted_1": int(cm_test[0][1])},
    "true_1": {"predicted_0": int(cm_test[1][0]), "predicted_1": int(cm_test[1][1])}
}

# Guardar matrices en el mismo archivo metrics.json
with open(metrics_path, "a") as f:  # append
    f.write(json.dumps(cm_train_dict) + "\n")
    f.write(json.dumps(cm_test_dict) + "\n")

print(f"✅ Matrices de confusión guardadas en: {metrics_path}")



✅ Matrices de confusión guardadas en: ../files/output/metrics.json
